In [2]:
import pandas as pd
import numpy as np
from copy import deepcopy

In [3]:
# read in processed in-house datasets, subset to targeting guides (i.e. all guides except intergenic and non-targeting controls)
file_path = '../../Data/primaryLibrary/'
annot_guide_lfcs_A549_Kox1 = pd.read_csv(file_path + 'A549_Kox1_zscored.csv')
annot_guide_lfcs_A549_Kox1 = annot_guide_lfcs_A549_Kox1[annot_guide_lfcs_A549_Kox1.Category.isin(['Essential', 'Non-essential'])]

annot_guide_lfcs_A549_Zim3 = pd.read_csv(file_path +  'A549_Zim3_zscored.csv')
annot_guide_lfcs_A549_Zim3 = annot_guide_lfcs_A549_Zim3[annot_guide_lfcs_A549_Zim3.Category.isin(['Essential', 'Non-essential'])]

annot_guide_lfcs_HCT116_Kox1 = pd.read_csv(file_path +  'HCT116_Kox1_zscored.csv')
annot_guide_lfcs_HCT116_Kox1 = annot_guide_lfcs_HCT116_Kox1[annot_guide_lfcs_HCT116_Kox1.Category.isin(['Essential', 'Non-essential'])]

annot_guide_lfcs_HCT116_Zim3 = pd.read_csv(file_path +  'HCT116_Zim3_zscored.csv')
annot_guide_lfcs_HCT116_Zim3 = annot_guide_lfcs_HCT116_Zim3[annot_guide_lfcs_HCT116_Zim3.Category.isin(['Essential', 'Non-essential'])]

#add column to specify domain before concatenating
annot_guide_lfcs_A549_Kox1['Domain'] = 'Kox1'
annot_guide_lfcs_A549_Zim3['Domain'] = 'Zim3'
annot_guide_lfcs_HCT116_Kox1['Domain'] = 'Kox1'
annot_guide_lfcs_HCT116_Zim3['Domain'] = 'Zim3'

InHouseCleanData = pd.concat([annot_guide_lfcs_A549_Kox1, annot_guide_lfcs_A549_Zim3, annot_guide_lfcs_HCT116_Kox1, annot_guide_lfcs_HCT116_Zim3])

# read in design file to append additional features
ess_noness = pd.read_csv(file_path + 'essentials_nonessentials_CRISPRi_tiling_designs_v2.csv')
ess_noness['chromosome'] = ess_noness['Reference Sequence'].apply(lambda x: int(x.split("NC_")[1].split(".")[0]) if pd.notnull(x) else x)
ess_noness = ess_noness[['Target Gene Symbol','Target Gene ID','Category','chromosome','sgRNA Sequence', 
           'sgRNA Context Sequence', 'TSS Position', 
       'sgRNA \'Cut\' Site TSS Offset', 'sgRNA \'Cut\' Position', 
            'Strand of Target','Strand of sgRNA', 'Orientation',
            'On-Target Ruleset','On-Target Efficacy Score']]
InHouseCleanData = pd.merge(InHouseCleanData, ess_noness)

InHouseCleanData.head()

sgRNA Sequence condition   avg_lfc  n_obs Target Gene Symbol  \
0  AAAAAAAAAAAGGGCAGAAG      A549 -0.376159      2            APOBEC1   
1  AAAAAAAAAAATTAGACCTC      A549 -0.021644      2             MRGPRD   
2  AAAAAAAAAAATTAGACCTC    HCT116 -0.257163      2             MRGPRD   
3  AAAAAAAAAAATTAGACCTC    HCT116 -0.277625      2             MRGPRD   
4  AAAAAAAAAACAGGACACAG      A549  0.031810      2              LARS2   

   Target Gene ID       Category  TSS Position  sgRNA 'Cut' Site TSS Offset  \
0           339.0  Non-essential     7665908.0                       -999.0   
1        116512.0  Non-essential    68980986.0                       -735.0   
2        116512.0  Non-essential    68980986.0                       -735.0   
3        116512.0  Non-essential    68980986.0                       -735.0   
4         23395.0      Essential    45388577.0                        667.0   

           sgRNA Context Sequence On-Target Ruleset  On-Target Efficacy Score  \
0  AAAAAAAAAAAAAAAGGGCAGAAGTGGACT   RS3seq-Chen2013                   -0.7248   
1  AAAAAAAAAAAAAAATTAGACCTCAGGGCA   RS3seq-Chen2013                   -0.6309   
2  AAAAAAAAAAAAAAATTAGACCTCAGGGCA   RS3seq-Chen2013                   -0.6309   
3  AAAAAAAAAAAAAAATTAGACCTCAGGGCA   RS3seq-Chen2013                   -0.6309   
4  CAGAAAAAAAAAAACAGGACACAGGGGAGA   RS3seq-Chen2013                    0.0140   

   z_scored_avg_lfc Domain  chromosome  sgRNA 'Cut' Position Strand of Target  \
0         -1.389004   Kox1        12.0             7666908.0                -   
1         -0.528140   Kox1        11.0            68981722.0                -   
2         -1.381448   Kox1        11.0            68981722.0                -   
3         -1.315569   Zim3        11.0            68981722.0                -   
4         -0.398339   Kox1         3.0            45389244.0                +   

  Strand of sgRNA Orientation  
0               -       sense  
1               +   antisense  
2               +   antisense  
3               +   antisense  
4               -   antisense

In [6]:
# read in cleaned Gilbert ricin data
file_path = '../../Data/externalData/'
GilbertRicinCleanData = pd.read_csv(file_path + 'Gilbert2014Cleaned.csv')
GilbertRicinCleanData['condition'] = 'K562'
GilbertRicinCleanData['Domain'] = 'Kox1'
GilbertRicinCleanData.head()

Target Gene Symbol chromosome strand targeted  \
0              ARCN1      chr11               +   
1              ARCN1      chr11               +   
2              ARCN1      chr11               +   
3              ARCN1      chr11               +   
4              ARCN1      chr11               +   

   PAM genomic coordinate [hg38]                     context seq  \
0                    118567638.0  GCACTTTCCTTGTTTACCTCTGGTAGGTTT   
1                    118568026.0  GCAACCCTGTAAGCATGCTTCTTGAGGAGT   
2                    118568080.0  GATGCTGCTTCTTCGGAAATATAACGGTAG   
3                    118568134.0  TAGCACCTTCCAGGGCTTCCAACTTGGATC   
4                    118568146.0  ATTGCCTTATTGTAGCACCTTCCAGGGCTT   

               guide id                                              oligo  \
0  ARCN1_w_118438354.27  cccttggagaaCCAcctTGTTGGCACTTTCCTTGTTTACCTCTGGT...   
1  ARCN1_w_118438742.27  cccttggagaaCCAcctTGTTGGCAACCCTGTAAGCATGCTTCTTG...   
2  ARCN1_w_118438796.24  cccttggagaaCCAcctTGTTGGCTGCTTCTTCGGAAATATAAGTT...   
3  ARCN1_w_118438850.25  cccttggagaaCCAcctTGTTGGCACCTTCCAGGGCTTCCAACTGT...   
4  ARCN1_w_118438862.24  cccttggagaaCCAcctTGTTGGCCTTATTGTAGCACCTTCCAGTT...   

             sgRNA sequence  zscoreRelativeNC   Avg_LFC  \
0  GCACTTTCCTTGTTTACCTCTGGT          0.250358 -0.134928   
1  GCAACCCTGTAAGCATGCTTCTTG          0.157641 -0.166789   
2     GCTGCTTCTTCGGAAATATAA          0.216340 -0.146618   
3    GCACCTTCCAGGGCTTCCAACT          0.320467 -0.110835   
4     GCCTTATTGTAGCACCTTCCA         -0.708019 -0.464269   

   sgRNA sequence in Context Target Gene Function  zscoreRelativeNC_signed  \
0                       True           Resistance                -0.250358   
1                       True           Resistance                -0.157641   
2                       True           Resistance                -0.216340   
3                       True           Resistance                -0.320467   
4                       True           Resistance                 0.708019   

   Avg_LFC_signed condition Domain  
0        0.134928      K562   Kox1  
1        0.166789      K562   Kox1  
2        0.146618      K562   Kox1  
3        0.110835      K562   Kox1  
4        0.464269      K562   Kox1

In [7]:
# read in cleaned Nunez data
NunezCleanData = pd.read_csv(file_path + 'Nunez2021Cleaned.csv')
NunezCleanData['condition'] = 'K562'
NunezCleanData['Domain'] = 'Kox1'
NunezCleanData.head()

ID Target Gene Symbol chromosome strand  \
0  ACTR6_+_100592059              ACTR6      chr12      +   
1  ACTR6_+_100592060              ACTR6      chr12      +   
2  ACTR6_+_100592066              ACTR6      chr12      +   
3  ACTR6_+_100592067              ACTR6      chr12      +   
4  ACTR6_+_100592092              ACTR6      chr12      +   

               sequence                     context_seq  hg38_coord  \
0  GCGAGCACATAGAGGGATCT  TGCTTCGAGCACATAGAGGGATCTGGGCCC   100198281   
1  GTCGAGCACATAGAGGGATC  CTGCTTCGAGCACATAGAGGGATCTGGGCC   100198282   
2  GCCTGCTTCGAGCACATAGA  GTCCACCTGCTTCGAGCACATAGAGGGATC   100198288   
3  GACCTGCTTCGAGCACATAG  GGTCCACCTGCTTCGAGCACATAGAGGGAT   100198289   
4  GAGACTCTGTCACCTCCGAG  GGGCTAGACTCTGTCACCTCCGAGGGGTCC   100198314   

   start_coord_hg38  end_coord_hg38  Phenotype scores-ave_Rep1_Rep2  \
0         100198284       100198303                       -0.018881   
1         100198285       100198304                        0.009292   
2         100198291       100198310                        0.000734   
3         100198292       100198311                       -0.008256   
4         100198317       100198336                        0.001314   

   Avg_LFC-Tfinal  z-score relative to NC condition Domain  
0        0.035104               -0.889828      K562   Kox1  
1        0.295731                0.580559      K562   Kox1  
2        0.214023                0.119582      K562   Kox1  
3        0.131427               -0.346402      K562   Kox1  
4        0.165469               -0.154343      K562   Kox1

In [8]:
# re-assigns sgRNA "cut" site in Gilbert and Nunez Datasets to be consistent with CRISPick
def CRISPickCutSite(df, strand_col, point_col, context_seq_col):
    CutSiteList = []
    for index, row in df.iterrows():
        if row[strand_col] == '+':
            CutSiteList.append(row[point_col] + 7)
        elif row[strand_col] == '-':
            CutSiteList.append(row[point_col] - 4)
        else:
            print(row[context_seq_col] + f'strand is neither + nor -')
    return CutSiteList

In [9]:
geneListGilbertAndNunez = GilbertRicinCleanData['Target Gene Symbol'].unique().tolist() + NunezCleanData['Target Gene Symbol'].unique().tolist()
pd.DataFrame(geneListGilbertAndNunez).to_csv(file_path + 'geneListGilbertAndNunez.csv', index = False)

Run CRISPick providing this list of genes as input to obtain MANE Select TSS positions. Link to output directory: https://portals.broadinstitute.org/gppx/crispick/platform/results/1da3859a-902d-471a-b06b-7aa43da31abd.

In [10]:
CRISPick_geneListGilbertAndNunez = pd.read_table(file_path + 'geneListGilbertAndNunez-sgrna-designs.txt')
CRISPick_geneListGilbertAndNunez = CRISPick_geneListGilbertAndNunez[[ 
       'Target Gene Symbol',  'Reference Sequence', 'Strand of Target',
       'TSS Position']]

In [11]:
#reassign cut site for Gilbert dataset guides
GilbertRicinCleanDataCutSite = CRISPickCutSite(GilbertRicinCleanData,'strand targeted' ,'PAM genomic coordinate [hg38]', 'context seq')
GilbertRicinCleanData['CRISPick CutSite'] = GilbertRicinCleanDataCutSite
GilbertRicinCleanData = pd.merge(GilbertRicinCleanData,CRISPick_geneListGilbertAndNunez, 
                                                 on = 'Target Gene Symbol')
GilbertRicinCleanData.head()

Target Gene Symbol chromosome strand targeted  \
0              ARCN1      chr11               +   
1              ARCN1      chr11               +   
2              ARCN1      chr11               +   
3              ARCN1      chr11               +   
4              ARCN1      chr11               +   

   PAM genomic coordinate [hg38]                     context seq  \
0                    118567638.0  GCACTTTCCTTGTTTACCTCTGGTAGGTTT   
1                    118568026.0  GCAACCCTGTAAGCATGCTTCTTGAGGAGT   
2                    118568080.0  GATGCTGCTTCTTCGGAAATATAACGGTAG   
3                    118568134.0  TAGCACCTTCCAGGGCTTCCAACTTGGATC   
4                    118568146.0  ATTGCCTTATTGTAGCACCTTCCAGGGCTT   

               guide id                                              oligo  \
0  ARCN1_w_118438354.27  cccttggagaaCCAcctTGTTGGCACTTTCCTTGTTTACCTCTGGT...   
1  ARCN1_w_118438742.27  cccttggagaaCCAcctTGTTGGCAACCCTGTAAGCATGCTTCTTG...   
2  ARCN1_w_118438796.24  cccttggagaaCCAcctTGTTGGCTGCTTCTTCGGAAATATAAGTT...   
3  ARCN1_w_118438850.25  cccttggagaaCCAcctTGTTGGCACCTTCCAGGGCTTCCAACTGT...   
4  ARCN1_w_118438862.24  cccttggagaaCCAcctTGTTGGCCTTATTGTAGCACCTTCCAGTT...   

             sgRNA sequence  zscoreRelativeNC   Avg_LFC  \
0  GCACTTTCCTTGTTTACCTCTGGT          0.250358 -0.134928   
1  GCAACCCTGTAAGCATGCTTCTTG          0.157641 -0.166789   
2     GCTGCTTCTTCGGAAATATAA          0.216340 -0.146618   
3    GCACCTTCCAGGGCTTCCAACT          0.320467 -0.110835   
4     GCCTTATTGTAGCACCTTCCA         -0.708019 -0.464269   

   sgRNA sequence in Context Target Gene Function  zscoreRelativeNC_signed  \
0                       True           Resistance                -0.250358   
1                       True           Resistance                -0.157641   
2                       True           Resistance                -0.216340   
3                       True           Resistance                -0.320467   
4                       True           Resistance                 0.708019   

   Avg_LFC_signed condition Domain  CRISPick CutSite Reference Sequence  \
0        0.134928      K562   Kox1       118567645.0       NC_000011.10   
1        0.166789      K562   Kox1       118568033.0       NC_000011.10   
2        0.146618      K562   Kox1       118568087.0       NC_000011.10   
3        0.110835      K562   Kox1       118568141.0       NC_000011.10   
4        0.464269      K562   Kox1       118568153.0       NC_000011.10   

  Strand of Target  TSS Position  
0                +     118572410  
1                +     118572410  
2                +     118572410  
3                +     118572410  
4                +     118572410

In [12]:
#calculate guide distance from MANE Select annotated TSS 
GilbertRicinCleanData["Distance to TSS"] = GilbertRicinCleanData.apply(
    lambda r:(r["CRISPick CutSite"] - r["TSS Position"]) if r["Strand of Target"] == "+" \
              else (r["TSS Position"] - r["CRISPick CutSite"]), axis=1)
GilbertRicinCleanData = GilbertRicinCleanData[abs(GilbertRicinCleanData['Distance to TSS']) <= 5000]
GilbertRicinCleanData.head()

Target Gene Symbol chromosome strand targeted  \
0              ARCN1      chr11               +   
1              ARCN1      chr11               +   
2              ARCN1      chr11               +   
3              ARCN1      chr11               +   
4              ARCN1      chr11               +   

   PAM genomic coordinate [hg38]                     context seq  \
0                    118567638.0  GCACTTTCCTTGTTTACCTCTGGTAGGTTT   
1                    118568026.0  GCAACCCTGTAAGCATGCTTCTTGAGGAGT   
2                    118568080.0  GATGCTGCTTCTTCGGAAATATAACGGTAG   
3                    118568134.0  TAGCACCTTCCAGGGCTTCCAACTTGGATC   
4                    118568146.0  ATTGCCTTATTGTAGCACCTTCCAGGGCTT   

               guide id                                              oligo  \
0  ARCN1_w_118438354.27  cccttggagaaCCAcctTGTTGGCACTTTCCTTGTTTACCTCTGGT...   
1  ARCN1_w_118438742.27  cccttggagaaCCAcctTGTTGGCAACCCTGTAAGCATGCTTCTTG...   
2  ARCN1_w_118438796.24  cccttggagaaCCAcctTGTTGGCTGCTTCTTCGGAAATATAAGTT...   
3  ARCN1_w_118438850.25  cccttggagaaCCAcctTGTTGGCACCTTCCAGGGCTTCCAACTGT...   
4  ARCN1_w_118438862.24  cccttggagaaCCAcctTGTTGGCCTTATTGTAGCACCTTCCAGTT...   

             sgRNA sequence  zscoreRelativeNC   Avg_LFC  ...  \
0  GCACTTTCCTTGTTTACCTCTGGT          0.250358 -0.134928  ...   
1  GCAACCCTGTAAGCATGCTTCTTG          0.157641 -0.166789  ...   
2     GCTGCTTCTTCGGAAATATAA          0.216340 -0.146618  ...   
3    GCACCTTCCAGGGCTTCCAACT          0.320467 -0.110835  ...   
4     GCCTTATTGTAGCACCTTCCA         -0.708019 -0.464269  ...   

   Target Gene Function zscoreRelativeNC_signed  Avg_LFC_signed  condition  \
0            Resistance               -0.250358        0.134928       K562   
1            Resistance               -0.157641        0.166789       K562   
2            Resistance               -0.216340        0.146618       K562   
3            Resistance               -0.320467        0.110835       K562   
4            Resistance                0.708019        0.464269       K562   

  Domain CRISPick CutSite  Reference Sequence Strand of Target TSS Position  \
0   Kox1      118567645.0        NC_000011.10                +    118572410   
1   Kox1      118568033.0        NC_000011.10                +    118572410   
2   Kox1      118568087.0        NC_000011.10                +    118572410   
3   Kox1      118568141.0        NC_000011.10                +    118572410   
4   Kox1      118568153.0        NC_000011.10                +    118572410   

   Distance to TSS  
0          -4765.0  
1          -4377.0  
2          -4323.0  
3          -4269.0  
4          -4257.0  

[5 rows x 21 columns]

In [13]:
#reassign cut site for Nunez dataset guides
NunezCleanDataCutSite = CRISPickCutSite(NunezCleanData,'strand' ,'hg38_coord', 'context_seq')
NunezCleanData['CRISPick CutSite'] = NunezCleanDataCutSite
NunezCleanData = pd.merge(NunezCleanData, CRISPick_geneListGilbertAndNunez,
                          on= ['Target Gene Symbol'],how = 'inner')
NunezCleanData.head()

ID Target Gene Symbol chromosome strand  \
0  ACTR6_+_100592059              ACTR6      chr12      +   
1  ACTR6_+_100592060              ACTR6      chr12      +   
2  ACTR6_+_100592066              ACTR6      chr12      +   
3  ACTR6_+_100592067              ACTR6      chr12      +   
4  ACTR6_+_100592092              ACTR6      chr12      +   

               sequence                     context_seq  hg38_coord  \
0  GCGAGCACATAGAGGGATCT  TGCTTCGAGCACATAGAGGGATCTGGGCCC   100198281   
1  GTCGAGCACATAGAGGGATC  CTGCTTCGAGCACATAGAGGGATCTGGGCC   100198282   
2  GCCTGCTTCGAGCACATAGA  GTCCACCTGCTTCGAGCACATAGAGGGATC   100198288   
3  GACCTGCTTCGAGCACATAG  GGTCCACCTGCTTCGAGCACATAGAGGGAT   100198289   
4  GAGACTCTGTCACCTCCGAG  GGGCTAGACTCTGTCACCTCCGAGGGGTCC   100198314   

   start_coord_hg38  end_coord_hg38  Phenotype scores-ave_Rep1_Rep2  \
0         100198284       100198303                       -0.018881   
1         100198285       100198304                        0.009292   
2         100198291       100198310                        0.000734   
3         100198292       100198311                       -0.008256   
4         100198317       100198336                        0.001314   

   Avg_LFC-Tfinal  z-score relative to NC condition Domain  CRISPick CutSite  \
0        0.035104               -0.889828      K562   Kox1         100198288   
1        0.295731                0.580559      K562   Kox1         100198289   
2        0.214023                0.119582      K562   Kox1         100198295   
3        0.131427               -0.346402      K562   Kox1         100198296   
4        0.165469               -0.154343      K562   Kox1         100198321   

  Reference Sequence Strand of Target  TSS Position  
0       NC_000012.12                +     100200816  
1       NC_000012.12                +     100200816  
2       NC_000012.12                +     100200816  
3       NC_000012.12                +     100200816  
4       NC_000012.12                +     100200816

In [14]:
#calculate guide distance from MANE Select annotated TSS 
NunezCleanData["Distance to TSS"] = NunezCleanData.apply(
    lambda r:(r["CRISPick CutSite"] - r["TSS Position"]) if r["Strand of Target"] == "+" \
              else (r["TSS Position"] - r["CRISPick CutSite"]), axis=1)
NunezCleanData = NunezCleanData[abs(NunezCleanData['Distance to TSS'])<=5000]
NunezCleanData.head()

ID Target Gene Symbol chromosome strand  \
0  ACTR6_+_100592059              ACTR6      chr12      +   
1  ACTR6_+_100592060              ACTR6      chr12      +   
2  ACTR6_+_100592066              ACTR6      chr12      +   
3  ACTR6_+_100592067              ACTR6      chr12      +   
4  ACTR6_+_100592092              ACTR6      chr12      +   

               sequence                     context_seq  hg38_coord  \
0  GCGAGCACATAGAGGGATCT  TGCTTCGAGCACATAGAGGGATCTGGGCCC   100198281   
1  GTCGAGCACATAGAGGGATC  CTGCTTCGAGCACATAGAGGGATCTGGGCC   100198282   
2  GCCTGCTTCGAGCACATAGA  GTCCACCTGCTTCGAGCACATAGAGGGATC   100198288   
3  GACCTGCTTCGAGCACATAG  GGTCCACCTGCTTCGAGCACATAGAGGGAT   100198289   
4  GAGACTCTGTCACCTCCGAG  GGGCTAGACTCTGTCACCTCCGAGGGGTCC   100198314   

   start_coord_hg38  end_coord_hg38  Phenotype scores-ave_Rep1_Rep2  \
0         100198284       100198303                       -0.018881   
1         100198285       100198304                        0.009292   
2         100198291       100198310                        0.000734   
3         100198292       100198311                       -0.008256   
4         100198317       100198336                        0.001314   

   Avg_LFC-Tfinal  z-score relative to NC condition Domain  CRISPick CutSite  \
0        0.035104               -0.889828      K562   Kox1         100198288   
1        0.295731                0.580559      K562   Kox1         100198289   
2        0.214023                0.119582      K562   Kox1         100198295   
3        0.131427               -0.346402      K562   Kox1         100198296   
4        0.165469               -0.154343      K562   Kox1         100198321   

  Reference Sequence Strand of Target  TSS Position  Distance to TSS  
0       NC_000012.12                +     100200816            -2528  
1       NC_000012.12                +     100200816            -2527  
2       NC_000012.12                +     100200816            -2521  
3       NC_000012.12                +     100200816            -2520  
4       NC_000012.12                +     100200816            -2495

### Combine datasets together

In [15]:
GilbertRicinCleanData = deepcopy(GilbertRicinCleanData[['Target Gene Symbol','chromosome','sgRNA sequence', 'context seq', 
                                'CRISPick CutSite', 'Strand of Target', 'TSS Position', 'Distance to TSS',
                                'zscoreRelativeNC_signed', 'Avg_LFC_signed', 
                                'condition', 'Domain']])


GilbertRicinCleanData.rename(columns={
                                      'sgRNA sequence':'sgRNA Sequence', 
                                      'context seq':'sgRNA Context Sequence',
                                      'CRISPick CutSite': 'sgRNA \'Cut\' Position', 
                                      'Distance to TSS':'sgRNA \'Cut\' Site TSS Offset'
                                     }, inplace = True)

In [16]:
NunezCleanData = deepcopy(NunezCleanData[['Target Gene Symbol','chromosome','sequence', 
                                               'context_seq', 'CRISPick CutSite',
                                                       'Strand of Target', 'TSS Position', 'Distance to TSS',
                                                       'Phenotype scores-ave_Rep1_Rep2','Avg_LFC-Tfinal', 
                         'z-score relative to NC','condition','Domain']])

NunezCleanData.rename(columns={'sequence':'sgRNA Sequence', 
                                    'context_seq':'sgRNA Context Sequence',
                                   'CRISPick CutSite': 'sgRNA \'Cut\' Position', 
                                      'Distance to TSS':'sgRNA \'Cut\' Site TSS Offset'}, inplace = True)

In [17]:
InHouseCleanData = deepcopy(InHouseCleanData\
                                 [['Target Gene Symbol','chromosome','sgRNA Sequence', 
                                   'sgRNA Context Sequence','sgRNA \'Cut\' Position',
                                   'Strand of Target', 'TSS Position', 'sgRNA \'Cut\' Site TSS Offset',
                                                       'z_scored_avg_lfc',
                                                              'condition','Domain']])



In [18]:
NunezCleanData['DataSet'] = 'Nunez'
GilbertRicinCleanData['DataSet'] = 'Gilbert'
InHouseCleanData['DataSet'] = 'InHouse'

In [19]:
DatasetCombine = pd.concat([NunezCleanData, GilbertRicinCleanData, InHouseCleanData]).reset_index(drop = True)
DatasetCombine.head()

Target Gene Symbol chromosome        sgRNA Sequence  \
0              ACTR6      chr12  GCGAGCACATAGAGGGATCT   
1              ACTR6      chr12  GTCGAGCACATAGAGGGATC   
2              ACTR6      chr12  GCCTGCTTCGAGCACATAGA   
3              ACTR6      chr12  GACCTGCTTCGAGCACATAG   
4              ACTR6      chr12  GAGACTCTGTCACCTCCGAG   

           sgRNA Context Sequence  sgRNA 'Cut' Position Strand of Target  \
0  TGCTTCGAGCACATAGAGGGATCTGGGCCC           100198288.0                +   
1  CTGCTTCGAGCACATAGAGGGATCTGGGCC           100198289.0                +   
2  GTCCACCTGCTTCGAGCACATAGAGGGATC           100198295.0                +   
3  GGTCCACCTGCTTCGAGCACATAGAGGGAT           100198296.0                +   
4  GGGCTAGACTCTGTCACCTCCGAGGGGTCC           100198321.0                +   

   TSS Position  sgRNA 'Cut' Site TSS Offset  Phenotype scores-ave_Rep1_Rep2  \
0   100200816.0                      -2528.0                       -0.018881   
1   100200816.0                      -2527.0                        0.009292   
2   100200816.0                      -2521.0                        0.000734   
3   100200816.0                      -2520.0                       -0.008256   
4   100200816.0                      -2495.0                        0.001314   

   Avg_LFC-Tfinal  z-score relative to NC condition Domain DataSet  \
0        0.035104               -0.889828      K562   Kox1   Nunez   
1        0.295731                0.580559      K562   Kox1   Nunez   
2        0.214023                0.119582      K562   Kox1   Nunez   
3        0.131427               -0.346402      K562   Kox1   Nunez   
4        0.165469               -0.154343      K562   Kox1   Nunez   

   zscoreRelativeNC_signed  Avg_LFC_signed  z_scored_avg_lfc  
0                      NaN             NaN               NaN  
1                      NaN             NaN               NaN  
2                      NaN             NaN               NaN  
3                      NaN             NaN               NaN  
4                      NaN             NaN               NaN

In [20]:
#how many context sequences appear across multiple datasets?
DatasetCombine.groupby('sgRNA Context Sequence')['DataSet'].nunique().value_counts()

1    204895
2      9728
3       364
Name: DataSet, dtype: int64

In [21]:
# looking specifically context sequences present in both in-house and other datasets to verify that cut position and cut site offset
    #were calculated correctly
datasets_per_seq = DatasetCombine.groupby('sgRNA Context Sequence')['DataSet'].nunique()
multi_dataset_seqs = datasets_per_seq.index[datasets_per_seq > 1]
multi_dataset_seq_df = DatasetCombine[DatasetCombine['sgRNA Context Sequence'].isin(multi_dataset_seqs)]
multi_dataset_seq_df = multi_dataset_seq_df[['Target Gene Symbol','sgRNA Context Sequence', 'Strand of Target','sgRNA \'Cut\' Site TSS Offset', 'DataSet']].drop_duplicates()
multi_dataset_seq_df.head()

Target Gene Symbol          sgRNA Context Sequence Strand of Target  \
3047              AURKB  AATTTGGAATTAATGACTAAATTGAGGTAA                -   
3048              AURKB  CCAGGGCAAGTCTAAATTACAATTTGGAAT                -   
3049              AURKB  AAGGATATAAAGTTACAGTTTAAAAGGCCC                -   
3050              AURKB  ATGATATCTGGGACTGAGCAGGCAGGGCCT                -   
3051              AURKB  AATGATATCTGGGACTGAGCAGGCAGGGCC                -   

      sgRNA 'Cut' Site TSS Offset DataSet  
3047                        826.0   Nunez  
3048                        806.0   Nunez  
3049                        778.0   Nunez  
3050                        674.0   Nunez  
3051                        673.0   Nunez

In [22]:
pivoted_df = multi_dataset_seq_df.pivot(index = ['sgRNA Context Sequence','Target Gene Symbol','Strand of Target'],
                                        columns = 'DataSet',
                                        values = 'sgRNA \'Cut\' Site TSS Offset').reset_index()
pivoted_df

DataSet          sgRNA Context Sequence Target Gene Symbol Strand of Target  \
0        AAAAAAAATTCTGCAGGAGGCAATAGGTTT              DDX18                +   
1        AAAAAAACGTTAAAGAAATTGTAAAGGAGA            DNTTIP2                -   
2        AAAAAATATATGCCATTTACCCCTCGGGAA             HEATR1                -   
3        AAAAAATTCCTGGAAAACCCAGGTAGGACT            METTL17                +   
4        AAAAAATTGAGGTCCTGAGTTAAGTGGCTG             MRPS25                -   
...                                 ...                ...              ...   
11159    TTTTTCATTGTTAAACCTGAAGACGGGGCA                TTK                +   
11160    TTTTTGGGCAGCACGGCTCCCATTCGGAAC             DNAJA3                +   
11161    TTTTTGTCGTAATCCTATTGAGTTCGGGTC              SNRPF                +   
11162    TTTTTTGCTCGCAGAAACAATACGTGGTCA               POP5                -   
11163    TTTTTTGTATCCGTGTTGGTTCTTAGGCGG               RPL4                -   

DataSet  Gilbert  InHouse  Nunez  
0            NaN    935.0  935.0  
1            NaN   -561.0 -562.0  
2            NaN   -554.0 -555.0  
3            NaN    617.0  617.0  
4            NaN   -423.0 -424.0  
...          ...      ...    ...  
11159        NaN    493.0  493.0  
11160        NaN   -887.0 -887.0  
11161        NaN   -752.0 -752.0  
11162        NaN   -535.0 -536.0  
11163        NaN    -58.0  -59.0  

[11164 rows x 6 columns]

It appears at first glance that the calculated TSS offset for the Nunez dataset is one off from the correct TSS offset for the in-house dataset (that was derived from CRISPick) ONLY when the guide is on the negative strand. Check whether this hypothesis is correct.

In [23]:
pivoted_df['InHouse - Nunez'] = pivoted_df['InHouse'] - pivoted_df['Nunez']
pivoted_df['InHouse - Gilbert'] = pivoted_df['InHouse'] - pivoted_df['Gilbert']
pivoted_df[['Strand of Target','InHouse - Nunez','InHouse - Gilbert']].value_counts()

Strand of Target  InHouse - Nunez  InHouse - Gilbert
+                 0.0              0.0                  277
-                 1.0              1.0                   87
dtype: int64

The calculated cut site is indeed one off for negative strand targeting guides.

In [24]:
#correct this offset
DatasetCombine.loc[(DatasetCombine['DataSet'].isin(['Nunez', 'Gilbert'])) & (DatasetCombine['Strand of Target'] == '-'), 
                      'sgRNA \'Cut\' Site TSS Offset'] += 1

#look at an example to verify whether it worked
DatasetCombine[DatasetCombine['sgRNA Context Sequence'] == 'AAAAAATTGAGGTCCTGAGTTAAGTGGCTG']

Target Gene Symbol chromosome        sgRNA Sequence  \
47174              MRPS25       chr3  GATTGAGGTCCTGAGTTAAG   
142713             MRPS25        3.0  AATTGAGGTCCTGAGTTAAG   
142714             MRPS25        3.0  AATTGAGGTCCTGAGTTAAG   
142715             MRPS25        3.0  AATTGAGGTCCTGAGTTAAG   
142716             MRPS25        3.0  AATTGAGGTCCTGAGTTAAG   

                sgRNA Context Sequence  sgRNA 'Cut' Position Strand of Target  \
47174   AAAAAATTGAGGTCCTGAGTTAAGTGGCTG            15065739.0                -   
142713  AAAAAATTGAGGTCCTGAGTTAAGTGGCTG            15065739.0                -   
142714  AAAAAATTGAGGTCCTGAGTTAAGTGGCTG            15065739.0                -   
142715  AAAAAATTGAGGTCCTGAGTTAAGTGGCTG            15065739.0                -   
142716  AAAAAATTGAGGTCCTGAGTTAAGTGGCTG            15065739.0                -   

        TSS Position  sgRNA 'Cut' Site TSS Offset  \
47174     15065315.0                       -423.0   
142713    15065315.0                       -423.0   
142714    15065315.0                       -423.0   
142715    15065315.0                       -423.0   
142716    15065315.0                       -423.0   

        Phenotype scores-ave_Rep1_Rep2  Avg_LFC-Tfinal  \
47174                        -0.019317        0.026164   
142713                             NaN             NaN   
142714                             NaN             NaN   
142715                             NaN             NaN   
142716                             NaN             NaN   

        z-score relative to NC condition Domain  DataSet  \
47174                -0.940264      K562   Kox1    Nunez   
142713                     NaN      A549   Kox1  InHouse   
142714                     NaN      A549   Zim3  InHouse   
142715                     NaN    HCT116   Kox1  InHouse   
142716                     NaN    HCT116   Zim3  InHouse   

        zscoreRelativeNC_signed  Avg_LFC_signed  z_scored_avg_lfc  
47174                       NaN             NaN               NaN  
142713                      NaN             NaN         -0.402571  
142714                      NaN             NaN         -0.761167  
142715                      NaN             NaN          0.075731  
142716                      NaN             NaN         -0.764584

### Remove overlapping genes
Guide RNAs which may target protein coding genes other than their intended target have the potential to confound analyses. Since the majority of guides within the combined tiling datasets are located within 1kb of the MANE Select TSS of their target gene, we remove all genes with any GENCODE annotated TSS within 1kb of any other GENCODE annotated TSS corresponding to a different gene.

In [25]:
# remove overlapping genes
overlapping_gene = pd.read_csv('../../Data/geneOverlap/GW_ensembl_protein_coding_df_1kb.csv')
overlapping_gene_2_3 =overlapping_gene[overlapping_gene['overlap type'].isin([2,3])]
gene_2_3_unique = set(list(overlapping_gene_2_3['Selected Gene name']) +\
list(overlapping_gene_2_3['Overlapping Gene name']))
NewDatasetCombine = deepcopy(DatasetCombine[~DatasetCombine['Target Gene Symbol'].isin(gene_2_3_unique)])

In [26]:
# did removing overlapping genes remove all multi-target guides?
conseq_with_varied_gene = NewDatasetCombine.groupby('sgRNA Context Sequence').filter(lambda NewDatasetCombine: NewDatasetCombine['Target Gene Symbol'].nunique() > 1)
#if yes, should be 0
len(conseq_with_varied_gene)

0

In [27]:
NewDatasetCombine['sgRNA Context Sequence'].nunique()

166624

In [28]:
len(NewDatasetCombine)

431095

In [29]:
#standardizes syntax with which chromosomes are labeled across datasets
def standardize_chromosome(chromosome):
    chromosome = str(chromosome).replace('.0', '')
    if not chromosome.startswith('chr'):
        chromosome = 'chr' + chromosome
    return chromosome

In [30]:
NewDatasetCombine['chromosome'] = NewDatasetCombine['chromosome'].apply(standardize_chromosome)

In [32]:
#define list of essential genes in in-house datasets
EssGene = ess_noness[ess_noness.Category == 'Essential']['Target Gene Symbol'].unique()

In [35]:
#remove non-essential genes
InhouseEss = NewDatasetCombine[(NewDatasetCombine.DataSet == 'InHouse') & (NewDatasetCombine['Target Gene Symbol'].isin(EssGene))]
other = NewDatasetCombine[~(NewDatasetCombine.DataSet == 'InHouse')]
newData = pd.concat([InhouseEss,other])
newData.value_counts(['DataSet', 'condition', 'Domain'])

DataSet  condition  Domain
Nunez    K562       Kox1      68575
InHouse  A549       Zim3      42609
         HCT116     Kox1      42609
         A549       Kox1      42570
         HCT116     Zim3      42544
Gilbert  K562       Kox1      18509
dtype: int64

In [36]:
out_path = '../../Data/cleanedData/'
newData.to_csv(out_path + 'CombinedCleanedDatasets.csv', index = False)